In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format
import json
import csv

### PARAMETRIA COMISIONES

In [2]:
parametria_comisiones = pd.read_csv(r'G:\My Drive\RESPALDO-PC\TOPUPS\PARAMETRIA TOPUPS.csv',dtype={'monto_recarga':float})
parametria_comisiones['inicio'] = pd.to_datetime(parametria_comisiones['inicio'])
parametria_comisiones['porcentaje_comision_final'] = parametria_comisiones['porcentaje_comision']/100
parametria_comisiones['index_comisiones'] = (parametria_comisiones['carrier'].astype(str).replace(' ', '',regex=True)+'-'
                                             +parametria_comisiones['monto_recarga'].astype(str))
parametria_comisiones.head()

,motivo_del_cambio,naturaleza,estatus,inicio,fin,carrier,monto_recarga,porcentaje_comision,categoria,porcentaje_comision_final,index_comisiones
0,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,30.00,7.00,MOVISTAR,0.07,MOVISTAR-30.0
1,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,50.00,7.00,MOVISTAR,0.07,MOVISTAR-50.0
2,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,100.00,7.00,MOVISTAR,0.07,MOVISTAR-100.0
3,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,200.00,7.00,MOVISTAR,0.07,MOVISTAR-200.0
4,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,500.00,7.00,MOVISTAR,0.07,MOVISTAR-500.0


In [3]:
parametria_comisiones.shape

(26, 11)

In [4]:
parametria_comisiones.dtypes

motivo_del_cambio                    object
naturaleza                           object
estatus                              object
inicio                       datetime64[ns]
fin                                  object
carrier                              object
monto_recarga                       float64
porcentaje_comision                 float64
categoria                            object
porcentaje_comision_final           float64
index_comisiones                     object
dtype: object

In [21]:
parametria_comisiones['carrier'].unique()

array(['MOVISTAR', 'UNEFON', 'TELCEL', 'ATT', 'IUSACELL'], dtype=object)

In [5]:
parametria_comisiones['index_comisiones'].unique()

array(['MOVISTAR-30.0', 'MOVISTAR-50.0', 'MOVISTAR-100.0',
       'MOVISTAR-200.0', 'MOVISTAR-500.0', 'UNEFON-30.0', 'UNEFON-50.0',
       'UNEFON-100.0', 'UNEFON-200.0', 'UNEFON-500.0', 'TELCEL-30.0',
       'TELCEL-50.0', 'TELCEL-80.0', 'TELCEL-100.0', 'TELCEL-200.0',
       'TELCEL-500.0', 'ATT-30.0', 'ATT-50.0', 'ATT-100.0', 'ATT-200.0',
       'ATT-500.0', 'IUSACELL-30.0', 'IUSACELL-50.0', 'IUSACELL-100.0',
       'IUSACELL-200.0', 'IUSACELL-500.0'], dtype=object)

### SERTI (EXTRAIDO EN EXCEL)

In [6]:
serti_files = glob.glob(r'G:\My Drive\RESPALDO-PC\TOPUPS\2021\10. OCTUBRE\SERTI\*.csv',recursive=False)
serti_files


['G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-01.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-16.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-15.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-14.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-13.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-12.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-11.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-10.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-09.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-08.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\SERTI\\serti 2021-10-07.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE

In [7]:
serti = pd.concat([pd.read_csv(f,dtype={'bin':object,'panTerminacion':object,'claveRechazo':object,'phone':object
                                        ,'transactionId':object,'importe':float})
                   .assign(file_name=os.path.basename(f)) for f in serti_files])                     
serti['last4_phone_serti'] = serti['phone'].str[-4:]
serti['serti_date'] = serti['date'].str[:10]
serti['serti_date'] = pd.to_datetime(serti['serti_date'])
serti.sort_values(by=['date'],inplace=True)
serti['INDEX_TOTAL'] = (serti['transactionId'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['last4_phone_serti'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['panTerminacion'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['importe'].astype(str).replace(' ', '',regex=True))
serti.reset_index(inplace=True)
serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,tarjetahabiente,comision,ivaComision,carrier,phone,folio,file_name,last4_phone_serti,serti_date,INDEX_TOTAL
0,9186,415231,3877,50.00,2021-10-01T00:00:32.281,44773598,Chip,"381,539.00",nan,0,...,TARJETAHABIENTE,0,0,Telcel,*******1921,"230,095.00",serti 2021-10-01.csv,1921,2021-10-01,44773598_1921_3877_50.0
1,9185,415231,1066,200.00,2021-10-01T00:00:50.411,44773603,Chip,"382,247.00",nan,0,...,TARJETAHABIENTE,0,0,Telcel,*******3234,"357,663.00",serti 2021-10-01.csv,3234,2021-10-01,44773603_3234_1066_200.0
2,9184,421316,8625,30.00,2021-10-01T00:00:50.464,44773605,Chip,"48,536.00",nan,0,...,TARJETAHABIENTE,0,0,Movistar,*******4097,"226,265.00",serti 2021-10-01.csv,4097,2021-10-01,44773605_4097_8625_30.0
3,9183,415231,1269,100.00,2021-10-01T00:00:56.282,44773607,Chip,"382,639.00",nan,0,...,TARJETAHABIENTE,0,0,Telcel,*******5106,"631,320.00",serti 2021-10-01.csv,5106,2021-10-01,44773607_5106_1269_100.0
4,9181,416916,0254,100.00,2021-10-01T00:01:08.415,44773612,Chip,"140,880.00",nan,0,...,TARJETAHABIENTE,0,0,Movistar,*******1693,"226,278.00",serti 2021-10-01.csv,1693,2021-10-01,44773612_1693_0254_100.0


In [8]:
serti.shape

(260472, 24)

In [9]:
duplicates_serti = serti[serti.duplicated(['transactionId'] , keep=False)]
duplicates_serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,tarjetahabiente,comision,ivaComision,carrier,phone,folio,file_name,last4_phone_serti,serti_date,INDEX_TOTAL


In [10]:
duplicates_serti.shape

(0, 24)

In [11]:
serti.drop_duplicates(subset='INDEX_TOTAL', keep='first', inplace=True)

In [12]:
serti.shape

(260472, 24)

In [13]:
serti.dtypes

index                         int64
bin                          object
panTerminacion               object
importe                     float64
date                         object
transactionId                object
entryMode                    object
numAuth                      object
terminal                    float64
caja                          int64
aprobada                       bool
claveRechazo                 object
merchant                      int64
producto                     object
tarjetahabiente              object
comision                      int64
ivaComision                   int64
carrier                      object
phone                        object
folio                       float64
file_name                    object
last4_phone_serti            object
serti_date           datetime64[ns]
INDEX_TOTAL                  object
dtype: object

In [14]:
serti.to_excel('SERTI.xlsx')

### PIVOT SERTI

In [15]:
pivot_serti = pd.pivot_table(serti,index=['serti_date']
                            ,columns=['aprobada']
                            ,values=['importe','transactionId']
                            ,aggfunc={'importe':np.sum,
                                     'transactionId':len},margins=True).reset_index()
pivot_serti.fillna(0,inplace=True)
pivot_serti.head()

serti_date   importe                       transactionId  \
aprobada                          False       True        All         False   
0         2021-10-01 00:00:00 60,170.00 542,380.00 602,550.00           924   
1         2021-10-02 00:00:00 51,340.00 531,880.00 583,220.00           799   
2         2021-10-03 00:00:00 42,180.00 395,550.00 437,730.00           631   
3         2021-10-04 00:00:00 83,410.00 721,580.00 804,990.00          1277   
4         2021-10-05 00:00:00 48,720.00 463,020.00 511,740.00           744   

                        
aprobada   True    All  
0          8263   9187  
1          8296   9095  
2          6163   6794  
3         10947  12224  
4          7205   7949

In [16]:
pivot_serti.shape

(32, 7)

### DB PRODUCTO

In [17]:
producto_files = glob.glob(r'G:\My Drive\RESPALDO-PC\TOPUPS\2021\**\DB\TOPUPS DB FACTURA*.csv',recursive=False)
producto_files


['G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\10. OCTUBRE\\DB\\TOPUPS DB FACTURA 2021.csv']

In [22]:
topups_db = pd.concat([pd.read_csv(f,dtype={'clip_external_tracking_id':object,'phone_number':object,
                                            'last4':object,'amount':float,'status_code':object})
                   .assign(file_name=os.path.basename(f)) for f in producto_files])
topups_db['clip_external_tracking_id'].fillna('sin_order_id_db',inplace=True)
topups_db['created_at'] = pd.to_datetime(topups_db['created_at'])
topups_db['updated_at'] = pd.to_datetime(topups_db['updated_at'])
topups_db['last4_phone'] = topups_db['phone_number'].str[-4:]
topups_db['INDEX_TOTAL'] = (topups_db['clip_external_tracking_id'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4_phone'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['amount'].astype(str).replace(' ', '',regex=True))
topups_db['index_comisiones'] = (topups_db['carrier'].astype(str).replace(' ', '',regex=True)+'-'
                                             +topups_db['amount'].astype(str))
topups_db.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (15,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,receipt_no,last4,authNum,orderId,type,response_code,file_name,last4_phone,INDEX_TOTAL,index_comisiones
0,dd5206b0-8515-4ba8-b517-132e9b77dcda,511d3a14-4fc4-46db-8ef8-f6f57dd7c517,33a02903-c95b-4639-96d8-800d1f86e79c,44773598,TELCEL,6623251921,2021-10-01,2021-10-01,2021-10-01 00:00:30,2021-10-01 00:00:30,...,TU-s0TG2Nh,3877,"381,539.00",41204784,emv_sale,00,TOPUPS DB FACTURA 2021.csv,1921,44773598_1921_3877_50.0,TELCEL-50.0
1,8df0219a-1040-4929-8b38-b19cd390bd1f,9b144863-8ed3-4b31-aed5-2ceac3af3fe4,758fed5f-a5c7-41fc-a6e2-3a7d78e83640,44773605,MOVISTAR,4491474097,2021-10-01,2021-10-01,2021-10-01 00:00:48,2021-10-01 00:00:48,...,TU-sef818M,8625,"48,536.00",41204790,emv_sale,00,TOPUPS DB FACTURA 2021.csv,4097,44773605_4097_8625_30.0,MOVISTAR-30.0
2,3e4a087e-f298-41ea-aa05-b61f35f46d60,c7257b88-9d97-47bc-8a23-c430fefb88ee,4c7d5adf-0ec3-42dd-9e73-849c8acc5f68,44773607,TELCEL,9381565106,2021-10-01,2021-10-01,2021-10-01 00:00:53,2021-10-01 00:00:53,...,TU-vAN1jnO,1269,"382,639.00",41204792,emv_sale,00,TOPUPS DB FACTURA 2021.csv,5106,44773607_5106_1269_100.0,TELCEL-100.0
3,c033b342-b036-4db1-b921-7d2a62653f61,ba2a4e06-32c6-4855-85c7-2b6e8b8b63f9,a9067acb-b092-4b75-a029-0628510dedc7,44773612,MOVISTAR,5534981693,2021-10-01,2021-10-01,2021-10-01 00:01:06,2021-10-01 00:01:06,...,TU-p0Jp597,0254,"140,880.00",41204796,emv_sale,00,TOPUPS DB FACTURA 2021.csv,1693,44773612_1693_0254_100.0,MOVISTAR-100.0
4,8a7af721-39ad-493c-84eb-5d49ec95186c,b53e5af9-5b72-4d96-9e85-3e4d6845371a,9279053b-410e-4bc0-8c0d-85f1e51db5ba,44773603,TELCEL,2721973234,2021-10-01,2021-10-01,2021-10-01 00:00:45,2021-10-01 00:00:45,...,TU-d0Um818,1066,"382,247.00",41204788,emv_sale,00,TOPUPS DB FACTURA 2021.csv,3234,44773603_3234_1066_200.0,TELCEL-200.0


In [23]:
topups_db.shape

(278664, 27)

In [24]:
topups_db['carrier'].unique()

array(['TELCEL', 'MOVISTAR', 'ATT', 'UNEFON'], dtype=object)

In [25]:
duplicates = topups_db[topups_db.duplicated(['clip_external_tracking_id'], keep=False)]
duplicates.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,receipt_no,last4,authNum,orderId,type,response_code,file_name,last4_phone,INDEX_TOTAL,index_comisiones
6,da4ed924-5091-4aeb-a876-fd8c64ab76f5,5a39bc48-fb09-44a4-adcb-eadf2f13bcbe,bf6e98fe-e457-4a4d-9cc1-6f1f7f44a1ed,sin_order_id_db,ATT,6461215663,2021-10-01,2021-10-01,2021-10-01 00:01:28,2021-10-01 00:01:28,...,TU-Z2i4Zrp,1893,NaN,41204798,emv_sale,H009,TOPUPS DB FACTURA 2021.csv,5663,sin_order_id_db_5663_1893_200.0,ATT-200.0
9,3ee649c0-e6d9-4492-ad0c-482e15b6fc70,1210a460-eab7-4322-af15-762c6c7efb13,15f402d9-1eae-47ea-884b-239c816896e4,sin_order_id_db,TELCEL,7713236589,2021-10-01,2021-10-01,2021-10-01 00:01:51,2021-10-01 00:01:51,...,TU-tSShuS5,7572,NaN,41204806,emv_sale,62,TOPUPS DB FACTURA 2021.csv,6589,sin_order_id_db_6589_7572_30.0,TELCEL-30.0
19,0952d88e-c307-4371-98ae-7de6b8398584,2b1e3abc-4131-4702-9aec-cfbcd0e1017b,17a7703e-feef-451f-9635-8130113389e5,sin_order_id_db,ATT,3323859948,2021-10-01,2021-10-01,2021-10-01 00:05:43,2021-10-01 00:05:43,...,TU-Dpnea9X,4369,NaN,41204855,emv_sale,68,TOPUPS DB FACTURA 2021.csv,9948,sin_order_id_db_9948_4369_100.0,ATT-100.0
22,b5ab702b-7481-49ce-8772-5d2b5a000af7,302c6ca8-6d65-4ad7-9b73-8c09f60f1cc4,df3a1373-4ad8-47d2-93d1-bdd022bd8bd1,sin_order_id_db,MOVISTAR,9982634649,2021-10-01,2021-10-01,2021-10-01 00:07:28,2021-10-01 00:07:28,...,TU-Z7PJgd9,5193,NaN,41204871,emv_sale,51,TOPUPS DB FACTURA 2021.csv,4649,sin_order_id_db_4649_5193_100.0,MOVISTAR-100.0
50,9505e71d-a955-49c3-8a12-1cca80dee256,5cdc8284-dbd5-47f1-b7a7-66e8f54cdbac,992ab446-d534-479d-a702-b268bfbf6255,sin_order_id_db,TELCEL,6311958885,2021-10-01,2021-10-01,2021-10-01 00:26:15,2021-10-01 00:26:15,...,TU-aAdFvuS,1820,NaN,41205040,emv_sale,5,TOPUPS DB FACTURA 2021.csv,8885,sin_order_id_db_8885_1820_50.0,TELCEL-50.0


In [26]:
duplicates.shape

(8172, 27)

In [27]:
duplicates['clip_external_tracking_id'].unique()

array(['sin_order_id_db', '0'], dtype=object)

In [28]:
topups_db.shape

(278664, 27)

In [29]:
topups_db.dtypes

merchant_id                          object
topup_id                             object
transaction_id                       object
clip_external_tracking_id            object
carrier                              object
phone_number                         object
created_at                   datetime64[ns]
updated_at                   datetime64[ns]
created_at_datetime                  object
updated_at_datetime                  object
payment_type                         object
status_code                          object
status_msg                           object
amount                              float64
processor                            object
issuer_auth_code                     object
folio_id                            float64
receipt_no                           object
last4                                object
authNum                              object
orderId                               int64
type                                 object
response_code                   

In [30]:
# topups_db.to_excel('DB_PRODUCT.xlsx')

### DB CON PARAMETRÍA DE COMISIONES

In [38]:
topups_db_final = topups_db.merge(parametria_comisiones[['index_comisiones','estatus','porcentaje_comision_final']]
                      ,on='index_comisiones',how='left')
topups_db_final['estatus'].fillna('Not_found',inplace=True)
topups_db_final['porcentaje_comision_final'].fillna(0,inplace=True)
topups_db_final['comision_db'] = topups_db_final['amount'] * topups_db_final['porcentaje_comision_final']
topups_db_final['iva_comision_db'] = topups_db_final['comision_db'] * 0.1600
topups_db_final.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,type,response_code,file_name,last4_phone,INDEX_TOTAL,index_comisiones,estatus,porcentaje_comision_final,comision_db,iva_comision_db
0,dd5206b0-8515-4ba8-b517-132e9b77dcda,511d3a14-4fc4-46db-8ef8-f6f57dd7c517,33a02903-c95b-4639-96d8-800d1f86e79c,44773598,TELCEL,6623251921,2021-10-01,2021-10-01,2021-10-01 00:00:30,2021-10-01 00:00:30,...,emv_sale,00,TOPUPS DB FACTURA 2021.csv,1921,44773598_1921_3877_50.0,TELCEL-50.0,VIGENTE,0.04,2.00,0.32
1,8df0219a-1040-4929-8b38-b19cd390bd1f,9b144863-8ed3-4b31-aed5-2ceac3af3fe4,758fed5f-a5c7-41fc-a6e2-3a7d78e83640,44773605,MOVISTAR,4491474097,2021-10-01,2021-10-01,2021-10-01 00:00:48,2021-10-01 00:00:48,...,emv_sale,00,TOPUPS DB FACTURA 2021.csv,4097,44773605_4097_8625_30.0,MOVISTAR-30.0,VIGENTE,0.07,2.10,0.34
2,3e4a087e-f298-41ea-aa05-b61f35f46d60,c7257b88-9d97-47bc-8a23-c430fefb88ee,4c7d5adf-0ec3-42dd-9e73-849c8acc5f68,44773607,TELCEL,9381565106,2021-10-01,2021-10-01,2021-10-01 00:00:53,2021-10-01 00:00:53,...,emv_sale,00,TOPUPS DB FACTURA 2021.csv,5106,44773607_5106_1269_100.0,TELCEL-100.0,VIGENTE,0.04,4.00,0.64
3,c033b342-b036-4db1-b921-7d2a62653f61,ba2a4e06-32c6-4855-85c7-2b6e8b8b63f9,a9067acb-b092-4b75-a029-0628510dedc7,44773612,MOVISTAR,5534981693,2021-10-01,2021-10-01,2021-10-01 00:01:06,2021-10-01 00:01:06,...,emv_sale,00,TOPUPS DB FACTURA 2021.csv,1693,44773612_1693_0254_100.0,MOVISTAR-100.0,VIGENTE,0.07,7.00,1.12
4,8a7af721-39ad-493c-84eb-5d49ec95186c,b53e5af9-5b72-4d96-9e85-3e4d6845371a,9279053b-410e-4bc0-8c0d-85f1e51db5ba,44773603,TELCEL,2721973234,2021-10-01,2021-10-01,2021-10-01 00:00:45,2021-10-01 00:00:45,...,emv_sale,00,TOPUPS DB FACTURA 2021.csv,3234,44773603_3234_1066_200.0,TELCEL-200.0,VIGENTE,0.04,8.00,1.28


In [39]:
topups_db_final.shape

(278664, 31)

### SERTI VS DB

In [40]:
cruce_1 = serti.merge(topups_db_final[['INDEX_TOTAL','topup_id','transaction_id','created_at','updated_at','status_code','status_msg'
                                 ,'amount','last4_phone','clip_external_tracking_id','carrier','type','response_code',
                                      'porcentaje_comision_final','comision_db','iva_comision_db']]
                      ,on='INDEX_TOTAL',how='left')
cruce_1['status_code'].fillna('Not_found',inplace=True)
cruce_1['topup_id'].fillna('Not_found',inplace=True)
cruce_1.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,status_msg,amount,last4_phone,clip_external_tracking_id,carrier_y,type,response_code,porcentaje_comision_final,comision_db,iva_comision_db
0,9186,415231,3877,50.00,2021-10-01T00:00:32.281,44773598,Chip,"381,539.00",nan,0,...,APPROVED,50.00,1921,44773598,TELCEL,emv_sale,00,0.04,2.00,0.32
1,9185,415231,1066,200.00,2021-10-01T00:00:50.411,44773603,Chip,"382,247.00",nan,0,...,APPROVED,200.00,3234,44773603,TELCEL,emv_sale,00,0.04,8.00,1.28
2,9184,421316,8625,30.00,2021-10-01T00:00:50.464,44773605,Chip,"48,536.00",nan,0,...,APPROVED,30.00,4097,44773605,MOVISTAR,emv_sale,00,0.07,2.10,0.34
3,9183,415231,1269,100.00,2021-10-01T00:00:56.282,44773607,Chip,"382,639.00",nan,0,...,APPROVED,100.00,5106,44773607,TELCEL,emv_sale,00,0.04,4.00,0.64
4,9181,416916,0254,100.00,2021-10-01T00:01:08.415,44773612,Chip,"140,880.00",nan,0,...,APPROVED,100.00,1693,44773612,MOVISTAR,emv_sale,00,0.07,7.00,1.12


In [41]:
cruce_1.shape

(260472, 39)

### TRUE ONLY

In [42]:
cruce_1_true = cruce_1[(cruce_1['aprobada'] == True)]
cruce_1_true['amount'].fillna(0,inplace=True)
cruce_1_true.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,status_msg,amount,last4_phone,clip_external_tracking_id,carrier_y,type,response_code,porcentaje_comision_final,comision_db,iva_comision_db
0,9186,415231,3877,50.00,2021-10-01T00:00:32.281,44773598,Chip,"381,539.00",nan,0,...,APPROVED,50.00,1921,44773598,TELCEL,emv_sale,00,0.04,2.00,0.32
1,9185,415231,1066,200.00,2021-10-01T00:00:50.411,44773603,Chip,"382,247.00",nan,0,...,APPROVED,200.00,3234,44773603,TELCEL,emv_sale,00,0.04,8.00,1.28
2,9184,421316,8625,30.00,2021-10-01T00:00:50.464,44773605,Chip,"48,536.00",nan,0,...,APPROVED,30.00,4097,44773605,MOVISTAR,emv_sale,00,0.07,2.10,0.34
3,9183,415231,1269,100.00,2021-10-01T00:00:56.282,44773607,Chip,"382,639.00",nan,0,...,APPROVED,100.00,5106,44773607,TELCEL,emv_sale,00,0.04,4.00,0.64
4,9181,416916,0254,100.00,2021-10-01T00:01:08.415,44773612,Chip,"140,880.00",nan,0,...,APPROVED,100.00,1693,44773612,MOVISTAR,emv_sale,00,0.07,7.00,1.12


In [43]:
cruce_1_true.shape

(236772, 39)

In [44]:
duplicates_cruce_1_true = cruce_1_true[cruce_1_true.duplicated(['INDEX_TOTAL'] , keep=False)]
duplicates_cruce_1_true.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,status_msg,amount,last4_phone,clip_external_tracking_id,carrier_y,type,response_code,porcentaje_comision_final,comision_db,iva_comision_db


In [45]:
duplicates_cruce_1_true.shape

(0, 39)

In [46]:
cruce_1_final = cruce_1_true[(cruce_1_true['topup_id'] != 'Not_found')]
cruce_1_final.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,status_msg,amount,last4_phone,clip_external_tracking_id,carrier_y,type,response_code,porcentaje_comision_final,comision_db,iva_comision_db
0,9186,415231,3877,50.00,2021-10-01T00:00:32.281,44773598,Chip,"381,539.00",nan,0,...,APPROVED,50.00,1921,44773598,TELCEL,emv_sale,00,0.04,2.00,0.32
1,9185,415231,1066,200.00,2021-10-01T00:00:50.411,44773603,Chip,"382,247.00",nan,0,...,APPROVED,200.00,3234,44773603,TELCEL,emv_sale,00,0.04,8.00,1.28
2,9184,421316,8625,30.00,2021-10-01T00:00:50.464,44773605,Chip,"48,536.00",nan,0,...,APPROVED,30.00,4097,44773605,MOVISTAR,emv_sale,00,0.07,2.10,0.34
3,9183,415231,1269,100.00,2021-10-01T00:00:56.282,44773607,Chip,"382,639.00",nan,0,...,APPROVED,100.00,5106,44773607,TELCEL,emv_sale,00,0.04,4.00,0.64
4,9181,416916,0254,100.00,2021-10-01T00:01:08.415,44773612,Chip,"140,880.00",nan,0,...,APPROVED,100.00,1693,44773612,MOVISTAR,emv_sale,00,0.07,7.00,1.12


In [47]:
cruce_1_final.shape

(235939, 39)

In [48]:
na_cruce_1_final = cruce_1_true[(cruce_1_true['topup_id'] == 'Not_found')]
na_cruce_1_final.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,status_msg,amount,last4_phone,clip_external_tracking_id,carrier_y,type,response_code,porcentaje_comision_final,comision_db,iva_comision_db
3092,6113,416916,3305,50.00,2021-10-01T12:27:52.454,44824893,Chip,"3,306.00",nan,0,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,nan,nan,nan
3098,6108,402766,9863,50.00,2021-10-01T12:28:09.349,44824932,Chip,"161,681.00",nan,0,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,nan,nan,nan
3215,5971,426807,6076,100.00,2021-10-01T12:37:30.492,44826057,Chip,"16,382.00",nan,0,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,nan,nan,nan
3945,5259,415231,8501,50.00,2021-10-01T13:39:49.984,44833179,Chip,"91,846.00",nan,0,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,nan,nan,nan
5164,4041,491282,5401,30.00,2021-10-01T15:35:09.578,44847432,Chip,"59,372.00",nan,0,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,nan,nan,nan


In [49]:
na_cruce_1_final.shape

(833, 39)

In [50]:
cruce_1_true.columns

Index(['index', 'bin', 'panTerminacion', 'importe', 'date', 'transactionId',
       'entryMode', 'numAuth', 'terminal', 'caja', 'aprobada', 'claveRechazo',
       'merchant', 'producto', 'tarjetahabiente', 'comision', 'ivaComision',
       'carrier_x', 'phone', 'folio', 'file_name', 'last4_phone_serti',
       'serti_date', 'INDEX_TOTAL', 'topup_id', 'transaction_id', 'created_at',
       'updated_at', 'status_code', 'status_msg', 'amount', 'last4_phone',
       'clip_external_tracking_id', 'carrier_y', 'type', 'response_code',
       'porcentaje_comision_final', 'comision_db', 'iva_comision_db'],
      dtype='object')

### PIVOT TRUE ONLY

In [52]:
pivot_true = pd.pivot_table(cruce_1_true,index=['serti_date']
                            ,columns=['status_code']
                            ,values=['importe','amount','transactionId','clip_external_tracking_id','comision_db',
                                     'iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                      'amount':np.sum,
                                      'comision_db':np.sum,
                                      'iva_comision_db':np.sum,
                                      'transactionId':len,
                                      'clip_external_tracking_id':len},margins=True).reset_index()
pivot_true.fillna(0,inplace=True)
pivot_true.head()

serti_date     amount                       \
status_code                               1 Not_found        All   
0            2021-10-01 00:00:00 541,820.00      0.00 541,820.00   
1            2021-10-02 00:00:00 531,270.00      0.00 531,270.00   
2            2021-10-03 00:00:00 395,070.00      0.00 395,070.00   
3            2021-10-04 00:00:00 717,750.00      0.00 717,750.00   
4            2021-10-05 00:00:00 462,640.00      0.00 462,640.00   

            clip_external_tracking_id                  comision_db            \
status_code                         1 Not_found    All           1 Not_found   
0                                8253     10.00   8253   27,955.70      0.00   
1                                8287      9.00   8287   27,333.60      0.00   
2                                6158      5.00   6158   20,094.90      0.00   
3                               10892     55.00  10892   37,187.10      0.00   
4                                7199      6.00   7199   23,735.20      0.00   

                         importe                      iva_comision_db  \
status_code       All          1 Not_found        All               1   
0           27,955.70 541,820.00    560.00 541,820.00        4,472.91   
1           27,333.60 531,270.00    610.00 531,270.00        4,373.38   
2           20,094.90 395,070.00    480.00 395,070.00        3,215.18   
3           37,187.10 717,750.00  3,830.00 717,750.00        5,949.94   
4           23,735.20 462,640.00    380.00 462,640.00        3,797.63   

                               transactionId                   
status_code Not_found      All             1 Not_found    All  
0                0.00 4,472.91          8253     10.00   8253  
1                0.00 4,373.38          8287      9.00   8287  
2                0.00 3,215.18          6158      5.00   6158  
3                0.00 5,949.94         10892     55.00  10892  
4                0.00 3,797.63          7199      6.00   7199

In [53]:
pivot_true.shape

(32, 19)

# EXCEL FINAL

In [54]:
writer = pd.ExcelWriter('CONCILIACION TOPUPS OCTUBRE COMISIONES 2021.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
pivot_serti.to_excel(writer, sheet_name='RESUMEN SERTI')
pivot_true.to_excel(writer, sheet_name='RESUMEN')
cruce_1_true.to_excel(writer, sheet_name='DETALLE COMPLETO',index=False)
cruce_1_final.to_excel(writer, sheet_name='FOUND',index=False)
na_cruce_1_final.to_excel(writer, sheet_name='NOT FOUND',index=False)
duplicates_cruce_1_true.to_excel(writer, sheet_name='DUPLICADOS_INDEX_TOTAL',index=False)
writer.save()
writer.close()